In [ ]:
#Importando as bibiotecas necessárias
from numpy import expand_dims, argmax

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [ ]:
from PIL import Image, ImageGrab

In [ ]:
import pygame
from pygame.locals import *

In [ ]:
#Carrega o modelo escolhido
my_model = load_model('Desenhe_ou_morra_model.h5')

In [ ]:
from random import randint, choice
from time import time

In [ ]:
pygame.init()
pygame.mixer.init()
pygame.font.init()

tela = pygame.display.set_mode((600, 400)) #Criando a tela do jogo (tamanho 600x400x)

pygame.display.set_caption('Desenhe ou Morra') #Título

preto = (0, 0, 0)

#Carrengando imagens
muro = pygame.image.load("imagens/muro.png")
limpar = pygame.image.load("imagens/limpar.png")
zombie = pygame.image.load("imagens/zombie.png")
sinal0 = pygame.image.load("imagens/sinal0.png")
sinal1 = pygame.image.load("imagens/sinal1.png")
sinal2 = pygame.image.load("imagens/sinal2.png")
sinal3 = pygame.image.load("imagens/sinal3.png")
sinal4 = pygame.image.load("imagens/sinal4.png")
tiro_imagem = pygame.image.load("imagens/tiro.png")
back = pygame.image.load("imagens/total.png")
back_end = pygame.image.load("imagens/tela_morte.png")
vida = pygame.image.load("imagens/vida.png")
pausa_imagem = pygame.image.load("imagens/pausa.png")

font_padrao = pygame.font.get_default_font()
font_zumbis = pygame.font.SysFont(font_padrao, 20)

#Carregando sons
tiro_som = pygame.mixer.Sound('sons/tiro.wav')
tiro_zumbie = pygame.mixer.Sound('sons/tiro_zumbie.wav')

imagens_sinais = (sinal0, sinal1, sinal2, sinal3, sinal4) #Armazenando as imagens dos sinais em uma tupla

todos_sinais = (0, 1, 2, 3, 4) #Números correspondentes a cada sinal
pontos_inicias = (600, 580, 560, 540, 520, 500, 480, 460, 440) #Possíveis coordenadas X em que um zumbi pode aparecer

Morte = False #Variável para verificar se o jogador morreu
Pause = False
PrimeiroClique = False #Variável para verificar se o jogador fez o primeiro clique
SegundoClique = False #Variável para verificar se o jogador fez o segundo clique


while True:
                     #---------------------------------------------------------
    zombies_150 = [] # Lista para armazenar as coordendas X de cada zumbi gerado  OBS: 150 são os zumbis que estão
    zombies_100 = [] #                                                                 na parte de baixo (coordenada Y = 150)
                     #---------------------------------------------------------        
    sinais_150 = []  # Lista para armazenas os sinais de cada zumbi                    100 são os zumbis que estão
    sinais_100 = []  #                                                                 na parte de cima (coordenada Y = 100)
                     #----------------------------------------------------------
    
    tela.blit(back, (-1, 0)) #Coloca a imagem back
    
    tela.blit(muro, (-10, 205)) #Coloca a imagem muro
    tela.blit(limpar, (207, 255)) #Coloca a imagem limpar (utilizada para limpar a área de desenho)
    
    maximo_zumbies = 3 #Máximo de zumbis permitidos na parte de cima e de baixo
    maximo_sinais = 3 #Máximo de sinais em cada zumbi
    velocidade = 10 #Número de pixels que os zumbis andam
    
    #Variáveis 
    mais = 0 
    x = 0    
    x1 = 0   
    x2 = 0
    y1 = 0
    y2 = 0
    
    vidas = 6 #Quantidade de vidas
    
    times = 0 #Variável para armazenar uma pseudo-contagem
    start_time = time() #Variável que armazena o tempo em que o jogo foi iniciado
    
    while vidas > 0: #Loop para o Jogo
        times += 1

        if len(zombies_150) == 0 and len(zombies_100) == 0: #Verifica se todos os zumbis foram mortos
            
            #Ifs utilizados para aumentar a dificuldade do jogo com o tempo
            if (time() - start_time) > 120: #Verifica se ja passaram 2 minutos desde a inicialização do Loop                 
                maximo_zumbies = 5                          
            elif (time() - start_time) > 240: #Verifica se ja passaram 4 minutos desde a inicialização do Loop           
                maximo_zumbies = 7                   
                maximo_sinais = 4
            elif (time() - start_time) > 360: #Verifica se ja passaram 6 minutos desde a inicialização do Loop
                maximo_sinais = 5
                velocidade = 20
            elif (time() - start_time) > 480: #Verifica se ja passaram 8 minutos desde a inicialização do Loop
                maximo_sinais = 6
                maximo_zumbies = 8
            elif (time() - start_time) > 600: #Verifica se ja passaram 10 minutos desde a inicialização do Loop
                maximo_zumbies = 9
                maximo_sinais = 7
            
            #For para criar os zumbies da parte de baixo e seus respectivos sinais
            for i in range(0, randint(1, maximo_zumbies)): 
                sinais_escolhidos = []
                zombies_150.append(choice(pontos_inicias)) #Escolhe um dos pontos iniciais
                for i2 in range(0, randint(1, maximo_sinais)):
                    sinais_escolhidos.append(choice(todos_sinais))
                sinais_150.append(sinais_escolhidos)
            
            #For para criar os zumbies da parte de cima e seus respectivos sinais
            for i in range(0, randint(1, maximo_zumbies)):
                sinais_escolhidos = []
                zombies_100.append(choice(pontos_inicias))
                for i2 in range(0, randint(1, maximo_sinais)):
                    sinais_escolhidos.append(choice(todos_sinais))
                sinais_100.append(sinais_escolhidos)
            
            zombies_100.sort() # Ordenar a lista para desenhar primeiro os que estiverem atrás (menor coordenada X)
            zombies_150.sort() # OBS: Mesmo perdendo os sinais corretos de cada zumbi gerado pelo for, não afetará o jogo.
            
        if times % 500 == 0: #If utilizado para a movimentação dos zumbis

            tela.blit(back, (-1, 0)) 
            
            #For utilizado para desenhar cada zumbi da parte de cima e seus respectivos sinais
            for i in range (-1, (len(zombies_100) + 1) * (-1), -1):
                mais = 0
                x = zombies_100[i] + 25 #Armazena a coordenada X do zumbi e adiciona 25 
                for sinal in sinais_100[i]: #For para percorrer cada sinal do zumbi e desenha-los.
                    tela.blit(imagens_sinais[sinal], (x + mais, 85))
                    mais += 23

                if zombies_100[i] > 40: #For para percorrer cada zumbi e desenha-los.
                    tela.blit(zombie, (zombies_100[i], 100))
                    zombies_100[i] -= velocidade
                else:
                    tela.blit(zombie, (zombies_100[i], 100))
            
            #For utilizado para desenhar cada zumbi da parte de baixo e seus respectivos sinais
            for i in range (-1, (len(zombies_150) + 1) * (-1), -1):
                mais = 0
                x = zombies_150[i] + 25 #Armazena a coordenada X do zumbi e adiciona 25 
                for sinal in sinais_150[i]: #For para percorrer cada sinal do zumbi e desenha-los.
                    tela.blit(imagens_sinais[sinal], (x + mais, 135))
                    mais += 23

                if zombies_150[i] > 60: #For para percorrer cada zumbi e desenha-los.
                    tela.blit(zombie, (zombies_150[i], 150))
                    zombies_150[i] -= velocidade
                else:
                    tela.blit(zombie, (zombies_150[i], 150))
                    
            #OBS: OS zumbis e seus sinais são desenhados do último ao primeiro
                
            tela.blit(muro, (-10, 205)) #Desenha o muro de baixo para que ele fique sobre os zumbis de baixo
            
        if times % 2000 == 0: #if representando as "mordidas" dos zumbis
            for i in range(0, len(zombies_150)): #Percorre cada zumbi de baixo
                if zombies_150[i] <= 60: #Verifica se o zumbi esta perto do solado
                    vidas -= 1           #se estiver, é retirado uma vida
                    
            if vidas == 0: #Se não tiver mais vidas, ele sai do loop(jogo acaba)
                break
            
            for i in range(0, len(zombies_100)):
                if zombies_100[i] <= 40:
                    vidas -= 1
            
            if vidas == 0:
                break
            
        mais = 0
        #For para desenhar os corações
        for i in range(0, vidas):
            tela.blit(vida, (mais, 10))
            mais += 35
        
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
            if event.type == pygame.MOUSEBUTTONDOWN: #Verifica se o jogador clicou

                x1, y1 = pygame.mouse.get_pos() #Pega a posição x e y do mouse na hora do clique

                if 205 <= x1 <= 395:       # Verifica se o mouse foi clicado 
                    if 255 <= y1 <= 400:   # dentro da área permitada para desenhar
                        if not PrimeiroClique:
                            PrimeiroClique = True
                            SegundoClique = False
                        else:
                            PrimeiroClique = False
                            SegundoClique = True
            
            if event.type == pygame.KEYDOWN: #Verifica se o jogador apertou alguma tecla
                if event.key == ord('p'): # Se apertou a tecla "p", o jogo será pausado
                    Pause = True
        
        if Pause:
            tela.blit(pausa_imagem, (220, 30))
            pygame.display.update()
        while Pause:
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                
                if event.type == pygame.KEYDOWN:
                    if event.key == ord('p'):
                        Pause = False
        
        if PrimeiroClique: #Verifica se o jogador ja deu primeiro clique
            x1, y1 = pygame.mouse.get_pos() #Pega a posição x e y do mouse na hora do clique
                        
            if 205 <= x1 <= 395:      # Verifica se o mouse está dentro 
                if 255 <= y1 <= 400:  # da área permitada para desenhar
                    pygame.draw.circle(tela, preto,(x1, y1),5, 0) #Se estiver, ele desenha um círculo preto
        
        if SegundoClique: #Verifica se o jogador ja deu o segundo clique
            img = ImageGrab.grab() #Tira print da tela

            size = img.size #Armazena o tamanho da tela
            x1 = size[0]/2-93; y1= size[1]/2+57 #calcular as coordenadas corretas da
            x2 = x1+188; y2 = y1+140            #área de desenho

            portion = img.crop((x1, y1, x2, y2)) #Pega a área desenho a partir das coordenadas

            img = portion.resize((32,32)) #Redimensiona para 32x32x
            img = img.convert('L') #Coverte para escala de cinza
            
            imagem = img_to_array(img) #Transforma a imagem em um array
            imagem = expand_dims(imagem, axis = 0) #Expande para 32x32x1x
            imagem /= 255 #Normaliza a imagem

            desenho = argmax(my_model.predict(imagem)) #Manda a imagem para o modelo e retorna a classificação
        
            for pos1, sinais in enumerate(sinais_100): #For para percorer os arrays de sinais dos zumbis de cima
                if sinais[0] == desenho: #Verifica se o desenho é igual ao primeiro sinal de cada zumbi
                    del sinais_100[pos1][0] #exclui o primeiro sinal
                    tiro_som.play()
                    tiro_som.set_volume(0.09)
                    tela.blit(tiro_imagem, (97, 118))
                    tiro_zumbie.play()

                if len(sinais_100[pos1]) == 0: #Se não existir mais sinais, o zumbi é destruído
                    del sinais_100[pos1]
                    del zombies_100[pos1]

            for pos1, sinais in enumerate(sinais_150):
                if sinais[0] == desenho:
                    del sinais_150[pos1][0]
                    tiro_som.play()
                    tiro_som.set_volume(0.09)
                    tela.blit(tiro_imagem, (118, 167))
                    tiro_zumbie.play()

                if len(sinais_150[pos1]) == 0:
                    del sinais_150[pos1]
                    del zombies_150[pos1]
            
            tela.blit(limpar, (207, 255)) # Limpa a área desenhada
            SegundoClique = False
        
        pygame.display.update()
    
    Morte = True
    tela.blit(back_end, (0, 0))
    pygame.display.update()
    
    while Morte:
        
        for event in pygame.event.get():
            
            if event.type == QUIT:
                pygame.quit()
                
            if event.type == pygame.KEYDOWN:
                if event.key == ord('s'):
                    Morte = False
                    
                elif event.key == ord('n'):
                    pygame.quit()
